In [7]:
import numpy as np
from functools import reduce

In [22]:
zero = np.array([1, 0])
one = np.array([0, 1])

In [3]:
zero2 = np.kron(zero, zero)

In [4]:
zero2

array([1, 0, 0, 0])

In [6]:
np.outer(zero, zero)

array([[1, 0],
       [0, 0]])

In [8]:
def zeron(n):
    '''
    Generate |00000...> state
    '''
    zero = np.array([1, 0])
    return reduce(np.kron, [zero]*n)

In [12]:
np.outer(zeron(2), zeron(2))

array([[1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [32]:
def bitState(bString):
    '''
    Generate bit string state
    
    Args
    ----
    bString : String of 0s and 1s describing state
    '''
    stateDict = {
        '0' : np.array([1, 0]),
        '1' : np.array([0, 1])
    }

    bArr = [stateDict[b] for b in bString]
    return reduce(np.kron, bArr)

def bitProjector(bString):
    '''
    Generate projector for given bitString state
    '''
    state = bitState(bString)
    return np.outer(state, state)

In [33]:
bitState('10')

array([0, 0, 1, 0])

In [34]:
np.kron(zero, one)

array([0, 1, 0, 0])

In [35]:
bitProjector('10')

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 0]])

Generate random 4 qubit state. Trace out the first two qubits and project on the last two. 

In [39]:
nqubits = 4
dim = 2**4
phi = np.random.rand(dim) + 1j*np.random.rand(dim) # Generate a random state
norm = np.linalg.norm(phi)
phi = phi / norm # normalise state

In [40]:
phi @ np.conj(phi)

(1.0000000000000002+0j)

In [42]:
import qiskit
import qiskit.quantum_info as qi

In [43]:
state = qi.Statevector(bitState('10'))

In [45]:
state.draw(output='latex')

<IPython.core.display.Latex object>

In [46]:
dMatrix = qi.DensityMatrix(state)

In [47]:
dMatrix

DensityMatrix([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
               [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
               [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
               [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
              dims=(2, 2))


In [48]:
qi.partial_trace(dMatrix, [0])

DensityMatrix([[0.+0.j, 0.+0.j],
               [0.+0.j, 1.+0.j]],
              dims=(2,))


In [49]:
qi.DensityMatrix(bitState('1'))

DensityMatrix([[0.+0.j, 0.+0.j],
               [0.+0.j, 1.+0.j]],
              dims=(2,))


In [69]:
def partialTrace(state, tracedInds):
    
    nqubits = int(np.log2(state.shape[0]))
    state = state.reshape([2]*nqubits)
    
    traced = np.tensordot(state, state.conj(), [tracedInds, tracedInds])

    nRemaining = len(traced.shape) // 2
    return traced.reshape([2**nRemaining]*2)

In [70]:
partialTrace(bitState('10'), [0])

array([[1, 0],
       [0, 0]])

Verifying the behaviour of the projector

In [72]:
state = np.random.randn(4) + 1j*np.random.rand(4)
state = state / np.linalg.norm(state)

In [73]:
phi = np.outer(state, state)

In [74]:
phi

array([[ 0.06851472+0.26107273j,  0.20143244+0.21700038j,
        -0.26204406-0.1517595j ,  0.07432868+0.1103321j ],
       [ 0.20143244+0.21700038j,  0.30715371+0.10555942j,
        -0.33198982-0.01108458j,  0.12900683+0.06821381j],
       [-0.26204406-0.1517595j , -0.33198982-0.01108458j,
         0.32793682-0.08874021j, -0.14660027-0.02800304j],
       [ 0.07432868+0.1103321j ,  0.12900683+0.06821381j,
        -0.14660027-0.02800304j,  0.05252363+0.03924983j]])

In [75]:
proj = bitProjector('00')

In [76]:
proj

array([[1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [77]:
np.trace(proj @ phi)

(0.06851471651927271+0.2610727310919041j)

In [78]:
proj @ phi

array([[ 0.06851472+0.26107273j,  0.20143244+0.21700038j,
        -0.26204406-0.1517595j ,  0.07432868+0.1103321j ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ]])

In [85]:
bin(4)[2:].zfill(5)

'00100'

In [87]:
bstrings = [bin(i)[2:].zfill(4) for i in range(10)]

for b in bstrings:
    print(b)
    print(np.argwhere(bitProjector(b)))

0000
[[0 0]]
0001
[[1 1]]
0010
[[2 2]]
0011
[[3 3]]
0100
[[4 4]]
0101
[[5 5]]
0110
[[6 6]]
0111
[[7 7]]
1000
[[8 8]]
1001
[[9 9]]
